In [2]:
import sagemaker
bucket = "tickercardiology-echocv-sagemaker"
key = "model-data"

bucket = "tickercardiology-echocv-sagemaker"
key = "model-data"
train_data = "s3://{}/{}/train/".format(bucket, key)
validation_data = "s3://{}/{}/test/".format(bucket, key)

# s3model = 's3://{}/model/'.format(bucket)
s3model = "vpmodel.tar.gz"
model_data = sagemaker.session.s3_input(s3model, 
                                        distribution= 'FullyReplicated',
                                        s3_data_type='S3Prefix',
                                        content_type='application/x-sagemaker-model')

data_channels = {'train': train_data, 'validation': validation_data, 'model': model_data}

In [5]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()

incr_ic = sagemaker.estimator.Estimator(data_channels['train'], 
                                        role, 
                                        train_instance_count=1, 
                                        train_instance_type='ml.m5.2xlarge', 
                                        train_volume_size = 20, 
                                        train_max_run= 5000, 
                                        input_mode= 'File', 
                                        output_path="temp", 
                                        sagemaker_session=sess)

incr_ic.set_hyperparameters(num_layers=18, 
                            image_shape= "224,224,3", 
                            num_classes=8, 
                            num_training_samples=1350, 
                            mini_batch_size=64, 
                            epochs=10, 
                            learning_rate=0.01, 
                            top_k=2)

incr_ic.fit(inputs=data_channels, logs=True)

ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: 2 validation errors detected: Value 'temp' at 'outputDataConfig.s3OutputPath' failed to satisfy constraint: Member must satisfy regular expression pattern: ^(https|s3)://([^/]+)/?(.*)$; Value 'vpmodel.tar.gz' at 'inputDataConfig.1.member.dataSource.s3DataSource.s3Uri' failed to satisfy constraint: Member must satisfy regular expression pattern: ^(https|s3)://([^/]+)/?(.*)$